This notebook enables to convert the pixel coordinates of each segment 50% included in polygons into its 730 vector values.  
The output is a csv/txt/ftr file that can be loaded as a dtaframe.  

***Import necessary libraries***

In [1]:
import os
from rasterio.plot import show
import numpy as np
import rasterio
import rasterio.features
import pandas as pd
import time
import tifffile as tiff
import random
#import feather

Lets create the files variables

In [2]:
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'
contained_all_segments = 'data/contained_all_segments.txt'

contain_small_20 = 'Prepro_Data/filtered_segments_20_ordered (1).txt'

# Input and Output dataframe

Create the function to load the segments.

In [3]:
def load_data_from_file(contained_file):
    # load the data
    segment_id = []
    polygon_id = []
    class_id = []
    pixels = []
    pixels_per = []
    
    with open(contained_file) as f:
        next(f)  # Skip the header line
        for line in f:
            data = line.strip().split(',')
            # retrieve segment id, polygon id, class id
            segment_id.append(data[0])
            polygon_id.append(data[1])
            class_id.append(data[2])
            # retrieve pixels
            pixel_str = line.split('"')[1]  # Extracting data between the first two quotes
            # Handle different representations of pixel data
            pixel_str = pixel_str.replace('), (', '|').strip('][').replace('(', '').replace(')', '')
            pixel_list = [list(map(int, point.split(', '))) for point in pixel_str.split('|')]
            pixels.append(pixel_list)
            
            pixel_per_str = line.split('"')[3] # Extracting data between the second two quotes
            pixel_per_str = pixel_per_str.replace('), (', '|').strip('][').replace('(', '').replace(')', '')
            pixel_per_list = [list(map(int, point.split(', '))) for point in pixel_per_str.split('|')]
            pixels_per.append(pixel_per_list)
            
        
    # create a dataframe
    df = pd.DataFrame()
    df['segment_id'] = segment_id
    df['polygon_id'] = polygon_id
    df['class_id'] = class_id
    df['pixels'] = pixels
    df['Perimeter Pixels'] = pixels_per
    
    return df


Load the segment dataframe.

In [4]:
#dataframe_segment = load_data_from_file(contained_file_50)
dataframe_segment =  load_data_from_file(contain_small_20)
print(dataframe_segment.head())

  segment_id polygon_id class_id  \
0          0          0        0   
1          1          0        0   
2          2          0        0   
3          3          0        0   
4          4          0        0   

                                              pixels  \
0  [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 0...   
1  [[0, 5], [0, 6], [0, 7], [0, 8], [1, 5], [1, 6...   
2  [[0, 9], [0, 10], [0, 11], [0, 12], [1, 9], [1...   
3  [[0, 13], [0, 14], [0, 15], [0, 16], [1, 13], ...   
4  [[0, 17], [0, 18], [0, 19], [1, 16], [1, 17], ...   

                                    Perimeter Pixels  
0  [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [4, 4...  
1  [[0, 5], [1, 5], [2, 5], [3, 5], [4, 5], [4, 8...  
2  [[0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 1...  
3  [[0, 13], [1, 13], [2, 13], [3, 13], [4, 13], ...  
4  [[0, 17], [1, 16], [2, 16], [3, 16], [4, 17], ...  


Create an output dataframe similar to the input except coordinates of each pixel is an empty list.

In [5]:
# create a new dataframe for output
dataframe_segment_output = load_data_from_file(contain_small_20)
# clone dataframe_segment_output['pixels'] to dataframe_segment_output['pixels_value']
dataframe_segment_output['pixels_value'] = dataframe_segment_output['pixels'].apply(lambda x: x[:])

# change each pixel of each segment to an empty list
for index_output, row_output in dataframe_segment_output.iterrows():
    for i_output, pixel_output in enumerate(row_output['pixels_value']):
        row_output['pixels_value'][i_output] = []
        
print(dataframe_segment_output.head())

  segment_id polygon_id class_id  \
0          0          0        0   
1          1          0        0   
2          2          0        0   
3          3          0        0   
4          4          0        0   

                                              pixels  \
0  [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 0...   
1  [[0, 5], [0, 6], [0, 7], [0, 8], [1, 5], [1, 6...   
2  [[0, 9], [0, 10], [0, 11], [0, 12], [1, 9], [1...   
3  [[0, 13], [0, 14], [0, 15], [0, 16], [1, 13], ...   
4  [[0, 17], [0, 18], [0, 19], [1, 16], [1, 17], ...   

                                    Perimeter Pixels  \
0  [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [4, 4...   
1  [[0, 5], [1, 5], [2, 5], [3, 5], [4, 5], [4, 8...   
2  [[0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 1...   
3  [[0, 13], [1, 13], [2, 13], [3, 13], [4, 13], ...   
4  [[0, 17], [1, 16], [2, 16], [3, 16], [4, 17], ...   

                                        pixels_value  
0  [[], [], [], [], [], [], [], [], [], [], [], [...  

The objective is to replace all of the coordinates with the actual 730 vector value of each pixel (for each acquisition of each ban : 73*10 = 730).

# Tests before processing

First lets create necessary variables.

In [6]:
folder = "/Users/thibaultgillard/Downloads"
list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']
list_band_files = [rf'{folder}/normalized_s2_2020_B{band}.tif' for band in list_bands]

Now we want to compare loading a raster image and then extract value for specific coordinates multiple and opening and retrive coordinates multiple times.

In [7]:
# open the tif file
band = '04'
file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
random_5_pixels = [random.randint(0, 10) for i in range(100)]

with rasterio.open(file_path) as src:
    
    # measure time taken to read values at coordinates for 5 pixels
        start_time = time.time()
        data = src.read(1)
        for pixel in random_5_pixels:
            x, y = pixel, pixel
            value = data[x, y]
            #print(value)
        print("Time taken to read values at coordinates for 5 pixels:", time.time() - start_time)
    
band = '03'
file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
random_5_pixels = [random.randint(0, 10) for i in range(100)]

with rasterio.open(file_path) as src:
    # measure time taken to read all values for 5 pixels
    start_time = time.time()
    for pixel in random_5_pixels:
        x, y = pixel, pixel
        value = src.read(1)[x][y]
        #print(value)
    print("Time taken to read all values for 5 pixels:", time.time() - start_time)


Time taken to read values at coordinates for 5 pixels: 4.127902984619141
Time taken to read all values for 5 pixels: 5.274499893188477


Fastest way is to store the raster image in a variable and then read coordinates out of it, as expected.  

# Conversion

Lets do the conversion.  
We will iterate throug all bands, and all acquisitions, and for each we iterate through all pixels of the segments.  
We then retrieve the value and append it to the output dataframe.  
This way is way faster than iterate through pixels and open the image for each of them.  

In [8]:
start = 1 # always 1
acquisition = 73 # 73
print(f"folder: {folder}")
print(f"list_bands: {list_bands}")
print(f"start")

# iterate through each band file
for band in list_bands:
    file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
    print(f"file_path: {file_path}")
    
    with rasterio.open(file_path) as src:
        # for each band file, iterate through all acquisition
        for period in range(start, acquisition+1):
            # for each acquisition, iterate through all segments
            print(f"period: {period}/{acquisition} in band {band}")
            band_acquisition = src.read(period)
            for index, row in dataframe_segment.iterrows():
                # for all segments, iterate through all pixels and add the
                print(index, end='\r')
                for i, pixel in enumerate(row['pixels']):
                    x, y = pixel
                    value = band_acquisition[x,y]
                    # add the pixel value in the output dataframe
                    dataframe_segment_output.at[index, 'pixels_value'][i].append(value)

folder: /Users/thibaultgillard/Downloads
list_bands: ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']
start
file_path: /Users/thibaultgillard/Downloads/normalized_s2_2020_B02.tif
period: 1/73 in band 02
period: 2/73 in band 02
period: 3/73 in band 02
period: 4/73 in band 02
period: 5/73 in band 02
period: 6/73 in band 02
period: 7/73 in band 02
period: 8/73 in band 02
period: 9/73 in band 02
period: 10/73 in band 02
period: 11/73 in band 02
period: 12/73 in band 02
period: 13/73 in band 02
period: 14/73 in band 02
period: 15/73 in band 02
period: 16/73 in band 02
period: 17/73 in band 02
period: 18/73 in band 02
period: 19/73 in band 02
period: 20/73 in band 02
period: 21/73 in band 02
period: 22/73 in band 02
period: 23/73 in band 02
period: 24/73 in band 02
period: 25/73 in band 02
period: 26/73 in band 02
period: 27/73 in band 02
period: 28/73 in band 02
period: 29/73 in band 02
period: 30/73 in band 02
period: 31/73 in band 02
period: 32/73 in band 02
period: 33/73 in b

Lets see what we got.

In [9]:
dataframe_segment_output.head()

,segment_id,polygon_id,class_id,pixels,Perimeter Pixels,pixels_value
0,0,0,0,"[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 0...","[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [4, 4...","[[0.47762346, 0.41975307, 0.36265433, 0.590277..."
1,1,0,0,"[[0, 5], [0, 6], [0, 7], [0, 8], [1, 5], [1, 6...","[[0, 5], [1, 5], [2, 5], [3, 5], [4, 5], [4, 8...","[[0.6566358, 0.65123457, 0.6458333, 0.7507716,..."
2,2,0,0,"[[0, 9], [0, 10], [0, 11], [0, 12], [1, 9], [1...","[[0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 1...","[[0.51929015, 0.46682099, 0.41435185, 0.621141..."
3,3,0,0,"[[0, 13], [0, 14], [0, 15], [0, 16], [1, 13], ...","[[0, 13], [1, 13], [2, 13], [3, 13], [4, 13], ...","[[0.525463, 0.49305555, 0.46141976, 0.6535494,..."
4,4,0,0,"[[0, 17], [0, 18], [0, 19], [1, 16], [1, 17], ...","[[0, 17], [1, 16], [2, 16], [3, 16], [4, 17], ...","[[0.5146605, 0.49845678, 0.48225307, 0.6195987..."


# Results

Now we can save the output dataframe in different formats.

In [8]:
# save as text file
output_file = ''
dataframe_segment_output.to_csv(output_file, sep='\t', index=False)

In [9]:
# save as Excel file
output_file = 'D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/post_process_final/post_processed_data.xlsx'
datafrmae_load.to_excel(output_file, index=False)

In [10]:
# save as csv file
output_file = '/Users/thibaultgillard/Documents/EPF/git/Data/post_processed_data_small20.csv'
dataframe_segment_output.to_csv(output_file)

In [40]:
# save as feather file
output_file = 'D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/post_process_final/post_processed_data.ftr'
readFrame.to_feather(output_file)

The fastest way to save and load the dataframe seems to be feather (see [here](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d)).  
It is also the smallest file size.

And we can load it back as follow.

In [4]:
input_file = 'D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/Output_Cassio/transfer_6352516_files_1d1978c6/post_processed_data.ftr'
readFrame = pd.read_feather(input_file, columns=None, use_threads=True)
readFrame.head()

,segment_id,polygon_id,class_id,pixels
0,367854,0.0,3.0,"[[0.44984567, 0.49691358, 0.40354937, 0.622685..."
1,367855,0.0,3.0,"[[0.55632716, 0.5578704, 0.50848764, 0.7075617..."
2,367856,0.0,3.0,"[[0.53780866, 0.5671296, 0.4699074, 0.69058645..."
3,369130,0.0,3.0,"[[0.5231481, 0.5532407, 0.44984567, 0.6705247,..."
4,363215,1.0,1.0,"[[0.38194445, 0.43904322, 0.24845679, 0.493055..."


The below code is to test if the loaded dataframe is the same as the original one.  
TO do so we compare a manual extraction with the values from the outputdataframe.  

In [11]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output"
list_bands_check = ["02", "03", "12"]
pixels_in_data = [[1,2],[2,3],[3,4],[4,5],[5,6]]
# get the values of the pixel from the dataframe_segment
pixels = []
pixels_values = []
for pixel in pixels_in_data:
    coords = dataframe_segment['pixels'][pixel[0]][pixel[1]]
    pixels.append(coords)
    values = [readFrame['pixels'][pixel[0]][pixel[1]][1], readFrame['pixels'][pixel[0]][pixel[1]][74], readFrame['pixels'][pixel[0]][pixel[1]][658]]
    pixels_values.append(values)
print(f"pixels to check: {pixels}")
print(f"Output values of pixels to check: {pixels_values}")

pixels to check: [[1145, 2927], [1145, 2932], [1150, 2926], [1132, 2932], [1138, 2931]]
Output values of pixels to check: [[0.5140496, 0.57886904, 0.5831894], [0.43636364, 0.54761904, 0.6200345], [0.71900827, 0.6770833, 0.66781807], [0.5553719, 0.71875, 0.6632124], [0.7123967, 0.8720238, 0.75187105]]


In [17]:
values_true_check = []
list_bands_check = ["02", "03", "12"]

# iterate through each band file
for band in list_bands_check:
    file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
    print(f"file_path: {file_path}")
    
    with rasterio.open(file_path) as src:
        period = 2
        
        band_acquisition = src.read(period)
        for pixel in pixels:
            x, y = pixel[0], pixel[1]
            value = band_acquisition[x,y]
            values_true_check.append(value)
print(f"Some of true values: {values_true_check}")

file_path: D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output\normalized_s2_2020_B02.tif
file_path: D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output\normalized_s2_2020_B03.tif
file_path: D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output\normalized_s2_2020_B12.tif
Some of true values: [0.5140496, 0.43636364, 0.71900827, 0.5553719, 0.7123967, 0.4827586, 0.4408867, 0.682266, 0.5, 0.6699507, 0.88459635, 0.79296684, 1.0, 0.7677068, 0.88261515]


The values are matching for random pixel, random bands and random acquisition, we can say with confidence that the conversion is working.  